## Data Cleaning and Processing

In [ ]:
# TODO 
pipeline! 
weighted_rating
count vectorize
fabric denstiy
fill na 
drop na
https://medium.com/@farukdemirok/hybrid-recommender-system-811ee411b697 - proceesubf dups

In [203]:
import numpy as np
import pandas as pd


In [204]:
df = pd.read_csv('data/patterns_df.csv')


df.drop(columns = ['pattern_id','name', 
                        'name_permalink','photos_url'], inplace= True)
df.head()

# # INCLUDE WEIGHTED AVERAGE SCORE RATHER THAN RATING?

,favorites_count,projects_count,difficulty_average,difficulty_count,rating_average,queued_projects_count,rating_count,pattern_type_names,pattern_type_clothing,pattern_needle_sizes,pattern_attributes,yardage_max,yardage,generally_available,gauge,gauge_divisor,free,downloadable,categories,yarn_weight_description
0,4526,2009,3.887821,936.0,4.466368,1404,892.0,shawl,True,"[{'id': 7, 'us': '7 ', 'metric': 4.5, 'us_stee...","['adult', 'lace', 'one-piece', 'top-down', 'tr...",NaN,874.0,2004/08/01 00:00:00 -0400,16.0,4.0,False,True,"['scarf', 'neck-torso', 'accessories', 'neck-t...",Fingering (14 wpi)
1,731,159,3.000000,78.0,4.266667,240,75.0,cardigan,True,"[{'id': 3, 'us': '3 ', 'metric': 3.25, 'us_ste...","['female', 'adult', 'no-ease', 'fitted', 'scoo...",1200.0,800.0,2007/01/12 00:10:07 -0500,22.0,4.0,False,True,"['scarf', 'neck-torso', 'accessories', 'neck-t...",DK / Sport
2,6626,2845,2.972950,1183.0,4.153497,1550,1101.0,child,True,"[{'id': 7, 'us': '7 ', 'metric': 4.5, 'us_stee...","['baby', 'newborn-size', 'toddler', 'child', '...",798.0,342.0,2004/10/01 00:00:00 -0400,21.0,4.0,True,True,"['scarf', 'neck-torso', 'accessories', 'neck-t...",DK (11 wpi)
3,11633,5127,4.911677,2355.0,4.471346,4107,2251.0,socks,True,"[{'id': 1, 'us': '1 ', 'metric': 2.25, 'us_ste...","['top-cuff-down', 'heel-flap', 'lace', 'kitche...",NaN,388.0,2005/12/01 00:00:00 -0500,8.0,1.0,True,True,"['scarf', 'neck-torso', 'accessories', 'neck-t...",Fingering (14 wpi)
4,3920,872,2.926714,423.0,4.287958,1185,382.0,hat,True,"[{'id': 8, 'us': '8 ', 'metric': 5.0, 'us_stee...","['adult', 'ribbed', 'textured', 'seamed', 'sho...",NaN,220.0,2007/01/12 00:58:37 -0500,20.0,4.0,True,True,"['scarf', 'neck-torso', 'accessories', 'neck-t...",Worsted (9 wpi)


In [193]:
df.columns

Index(['favorites_count', 'projects_count', 'difficulty_average',
       'difficulty_count', 'rating_average', 'queued_projects_count',
       'rating_count', 'pattern_type_names', 'pattern_type_clothing',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'generally_available', 'gauge', 'gauge_divisor', 'free', 'downloadable',
       'categories', 'yarn_weight_description'],
      dtype='object')

#### Categorical

In [8]:
cat_columns = ['pattern_type_clothing','downloadable',
               'free','categories', 'pattern_type_names','pattern_attributes','yarn_weight_description']
# THERE IS A PATTERN TYPE ID _ JUST USE THAT?? instead of names??
# MAYBE REGROUP AND ONE HOT ENCODE? OR READJUST AND LABEL ENCODE IDS?

In [111]:
# PATTERN_TYPES (ID AND NAMES)
print(pat.pattern_type_id.unique())
print(pat['pattern_type_names'].unique())
len(pat.pattern_type_names.unique())
# one hot encode?

[10  7 17  2  3  1  9  6  8 20 16  4 19 15  5 11 21 13 18]
['shawl' 'cardigan' 'child' 'socks' 'hat' 'scarf' 'other' 'pet' 'gloves'
 'shrug' 'toys' 'pullover' 'jacket' 'bag' 'baby' 'blanket' 'home' 'vest'
 'tee']


19

In [ ]:
# columns to one_hot encode, or true, false?
# There are 19 pattern_type_names - not sure which one to use...
cols_to_one_hot = ['pattern_type_clothing','downloadable','free','pattern_type_names'] # note pattern_type_names 19
cols_to_count_vectorize = ['categories', 'pattern_type_names','pattern_attributes']
cols_to_transform = ['yarn_weight_description']

In [205]:
# YARN WEIGHTS
# convert to yarn weights correspnding actual relative thicknesses 
# yarn_weight_id are not sequential nor ordered, so can't use those
# (indstry standards splits into 7 categories, but this goes more ganually

def encode_yarn_weights(df):
    """ uses yarn_weight_description to convert weights correspnding actual relative thicknesses """
    yarn_weights = {'Lace' : 1,
                    'Fingering (14 wpi)': 2,
                    'Sport (12 wpi)': 3,
                    'DK / Sport' : 4,
                    'DK (11 wpi)' : 5,
                    'Worsted (9 wpi)':6,
                    'Aran / Worsted': 7,
                    'Aran (8 wpi)': 8,
                    'Bulky (7 wpi)':9,
                    'Super Bulky (5-6 wpi)':10}
    try:
        df.replace({'yarn_weight_description':yarn_weights}, inplace=True)
        df.rename(columns = {'yarn_weight_description':'yarn_weight_code'}, inplace=True)
    except:
        print("okay - check out yarn_weight_description, something went wrong with the encoding")
    return df

df = encode_yarn_weights(df)

#### Numerical Columns

In [23]:
patt.columns
num_columns = ['favorites_count', 'projects_count', 'difficulty_average',
       'difficulty_count', 'rating_average', 'rating_count', 
        'pattern_needle_sizes', 'yardage', 'yardage_max',
       'generally_available', 'published', 'gauge',
       'gauge_divisor', 
       'yarn_weight_description', 'yarn_weight_id', 'yarn_weight_name',
       'yarn_weight_ply', 'yarn_weight_wpi', 'yarn_weight_knit_gauge']

# 'pattern_type_id', - maybe bring this back in after

In [28]:
# patt.loc[:,num_columns]

In [206]:
# YARDAGE

def fill_max_yardage(df):
    df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
    return df

df = get_yardage(df)

def use_avg_yardage(df):
    df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
    df['yardage_avg'] = (df['yardage_max'] + df['yardage_max'])/2
#     drop rows?
    return df

df = use_avg_yardage(df)


# FUNCTION THAT FOR EACH TYPE< IF MISSING < then replaces with avg
# else, drop row

In [ ]:
def get_weighted_rating(df):
    pass
    

In [44]:
num_df = patt[num_columns]

In [207]:
def consolidate_gauge(df):
    """ takes in gauge columns and normalizes them all to stiches per inch """
    try:
        df['gauge_per_inch'] = df.loc[:,'gauge']/df.loc[:,'gauge_divisor']
        df.drop(columns = ['gauge', 'gauge_divisor'], axis = 1, inplace = True)
    except:
        print("Error occured when consolidating gauge")
    return df

df =consolidate_gauge(df)

In [208]:
def split_date_released(df):
    """ takes in 'generally_available' column and splits out into month and year 
    (to take into account seasonal purchases, and yearly impacts)"""
    try:
        df['generally_available'] = pd.to_datetime(df['generally_available'], utc=True)
        df['month_avail'] = df.generally_available.dt.month
        df['year_avail'] = df.generally_available.dt.year
     
        df.drop(columns = ['generally_available'], axis = 1, inplace = True)
    except:
        "print - Error occured when trying to split dates"
    return df

df = split_date_released(df)

In [169]:
df.columns

Index(['pattern_id', 'name', 'name_permalink', 'favorites_count',
       'projects_count', 'difficulty_average', 'difficulty_count',
       'rating_average', 'queued_projects_count', 'rating_count',
       'pattern_type_names', 'pattern_type_clothing', 'photos_url',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'free', 'downloadable', 'categories', 'yarn_weight_description',
       'yardage_avg', 'gauge_per_inch', 'month_avail', 'year_avail'],
      dtype='object')

In [209]:
# NEEDLE SIZE
"""Get the list of metric needle sizes for knitting (can have several needle sizes for various parts of the pattern 
    and crochet hooks for finishing.  Generally the largest needle is the 
    one used for the main body (or largest portion of the project) """
import ast

def get_list_metric_needles(x):
    n=[]
    for i in range(len(x)):
        if x[i]['knitting'] == True:
            n.append(x[i]['metric'])
    return n

def get_needle_size(df):
    try:
        df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
    except:
        pass
    df['pattern_needle_sizes'] = df['pattern_needle_sizes'].apply(lambda x: get_list_metric_needles(x))
    df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: max(x) if (len(x)>=1) else None)
    return df

df = get_needle_size(df)

In [212]:
def create_fabric_density_ratio(df):
    df['fabric_density'] = df['yarn_weight_code']/df['pattern_needle_sizes']
    return(df)
#     ratio of need to needle size and yarn_weight
df =create_fabric_density_ratio(df)


In [213]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
favorites_count,4526,731,6626,11633,3920,1200,526,240,1057,762,...,209,284,2872,18092,2150,2583,2603,7658,163,395
projects_count,2009,159,2845,5127,872,104,43,40,1067,76,...,51,65,640,7774,483,326,965,742,9,347
difficulty_average,3.88782,3,2.97295,4.91168,2.92671,5.56604,3.15789,2.625,1.90079,4.18519,...,2.72727,1.64516,3.89209,2.34056,3.01136,4.31953,1.95604,2.37586,1.66667,2.05263
difficulty_count,936,78,1183,2355,423,53,19,8,504,27,...,22,31,278,3550,264,169,455,290,3,190
rating_average,4.46637,4.26667,4.1535,4.47135,4.28796,4.39535,4.25,2.875,3.93363,3.93548,...,3.47368,4.51852,4.24111,4.52574,4.43697,4.17341,4.44059,4.09091,3.33333,4.22754
queued_projects_count,1404,240,1550,4107,1185,277,178,23,283,242,...,61,72,766,3973,598,733,681,1979,18,99
rating_count,892,75,1101,2251,382,43,20,8,452,31,...,19,27,253,3186,238,173,404,264,3,167
pattern_type_names,shawl,cardigan,child,socks,hat,scarf,cardigan,shawl,scarf,shawl,...,cardigan,home,gloves,scarf,scarf,gloves,shawl,scarf,toys,scarf
pattern_type_clothing,True,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,False,True
pattern_needle_sizes,4.5,4,4.5,2.25,5,3.75,3.5,NaN,6,4,...,5,6,3.25,5,5,3.25,8,3.25,6.5,5.5


In [ ]:

def create_average_rating(df):
    pass

In [ ]:
process difficulty and rating average down to 3 decimal places

#### columns I'm unsure about:
Downloadable - do I keep?
Make sure yarn weight is filled - which one?? and related it back to yarn weight name
** fix pattern needles size
row gauge?? is this needed?? probably not, other gauge should be fine
** need a function to impute missing values!! figure out what should be for each row
** if lots on a particular row, drop


In [ ]:
# 

#### Get pattern page link

In [ ]:
link = f'https://www.ravelry.com/patterns/library/{permalink}'